In [26]:
import pandas as pd
import numpy as np
import numpy_financial as npf

class ModelFairCredit:
    def __init__(self, transaction_data, minimum_allowable_price=500, applicant_ratio=0.5, period_term=10, monthly_interest=0.01):
        self.transaction_data = transaction_data
        self.minimum_allowable_price = minimum_allowable_price
        self.applicant_ratio = applicant_ratio
        self.period_term = period_term
        self.monthly_interest = monthly_interest
        
        self.aggregated_amortization_schedule = pd.DataFrame()
        
    def generate_amortization_schedule(self):
        
        for _, row in self.transaction_data.iterrows():
        
            if row['Revenue'] < self.minimum_allowable_price:
                continue 
            
            if np.random.rand() > self.applicant_ratio:
                continue
            
            
            
            amortization_schedule = pd.DataFrame()
            amortization_schedule['Period'] = pd.date_range(start=row['Period'], periods=self.period_term, freq='30D')
            
            monthly_payment = npf.pmt(self.monthly_interest, self.period_term, -row['Revenue'])
            
            amortization_schedule['Revenue'] = monthly_payment
            amortization_schedule['Expense'] = 0
            amortization_schedule['Customer ID'] = row['Customer ID']
            
            
            # set the original row in self.transaction_data to 0 on Revenue column
            self.transaction_data.loc[self.transaction_data['Customer ID'] == row['Customer ID'], 'Revenue'] = 0
            
            # conct the amortization schedule to the aggregated amortization schedule
            self.aggregated_amortization_schedule = pd.concat([self.aggregated_amortization_schedule, amortization_schedule], ignore_index=True)
            
            return self.aggregated_amortization_schedule
        
    def group_by_period(self):
        
        # return after grouping by month-year
        return self.aggregated_amortization_schedule.groupby(self.aggregated_amortization_schedule['Period'].dt.to_period("M")).agg({'Revenue':'sum', 'Expense':'sum'}).reset_index()
        
        
     

In [27]:
transction_data = pd.read_csv('forecast_df_treatment.csv')[:100]

In [28]:
model = ModelFairCredit(transction_data)

model.generate_amortization_schedule()

,Period,Revenue,Expense,Customer ID
0,2025-01-25,151.721444,0,Patient 676
1,2025-02-24,151.721444,0,Patient 676
2,2025-03-26,151.721444,0,Patient 676
3,2025-04-25,151.721444,0,Patient 676
4,2025-05-25,151.721444,0,Patient 676
5,2025-06-24,151.721444,0,Patient 676
6,2025-07-24,151.721444,0,Patient 676
7,2025-08-23,151.721444,0,Patient 676
8,2025-09-22,151.721444,0,Patient 676
9,2025-10-22,151.721444,0,Patient 676


In [29]:
model.transaction_data[model.transaction_data['Customer ID'] == 'Patient 676']

,Period,Treatment,Revenue,Expense,Customer ID
79,2025-01-25,233,0.0,152.97,Patient 676


In [5]:
import pandas as pd
import numpy as np
import numpy_financial as npf

# Load the uploaded transaction data
file_path = 'forecast_df_treatment.csv'
transaction_data = pd.read_csv(file_path)

# Ensure correct data types for the model to work
transaction_data['Period'] = pd.to_datetime(transaction_data['Period'])
transaction_data['Revenue'] = transaction_data['Revenue'].astype(float)
transaction_data['Customer ID'] = transaction_data['Customer ID'].astype(str)

# Define the ModelFairCredit class
class ModelFairCredit:
    def __init__(self, transaction_data, minimum_allowable_price=500, applicant_ratio=0.5, period_term=10, monthly_interest=0.01, percentage_downpayment=0.3):
        self.transaction_data = transaction_data
        self.minimum_allowable_price = minimum_allowable_price
        self.applicant_ratio = applicant_ratio
        self.period_term = period_term
        self.monthly_interest = monthly_interest
        self.percentage_downpayment = percentage_downpayment
        
        self.aggregated_amortization_schedule = pd.DataFrame()
        
    def generate_amortization_schedule(self):
        adjusted_original_transaction = self.transaction_data.copy()
        
        for _, row in self.transaction_data.iterrows():
            if row['Revenue'] < self.minimum_allowable_price:
                continue 
            
            if np.random.rand() > self.applicant_ratio:
                continue
            
            amortization_schedule = pd.DataFrame()
            amortization_schedule['Period'] = pd.date_range(start=row['Period'], periods=self.period_term, freq='30D')
            
            price_as_principal = row['Revenue'] * (1 - self.percentage_downpayment)
            
            monthly_payment = npf.pmt(self.monthly_interest, self.period_term, -price_as_principal)
            
            amortization_schedule['Revenue'] = monthly_payment
            amortization_schedule['Expense'] = 0
            amortization_schedule['Customer ID'] = row['Customer ID']
            
            # Adjust the original transaction data directly for the current customer
            adjusted_original_transaction.loc[
                adjusted_original_transaction['Customer ID'] == row['Customer ID'], 'Revenue'
            ] = row['Revenue'] * self.percentage_downpayment
            
            # Concatenate the amortization schedule to the aggregated amortization schedule
            self.aggregated_amortization_schedule = pd.concat([self.aggregated_amortization_schedule, amortization_schedule], ignore_index=True)
        
        return self.aggregated_amortization_schedule, adjusted_original_transaction
        
    def group_by_period(self):
        # Return after grouping by month-year
        return self.aggregated_amortization_schedule.groupby(self.aggregated_amortization_schedule['Period'].dt.to_period("M")).agg({'Revenue':'sum', 'Expense':'sum'}).reset_index()

# Initialize the model and run the function
model = ModelFairCredit(transaction_data)
aggregated_amortization_schedule, adjusted_original_transaction = model.generate_amortization_schedule()

# Check the sums of the Revenue columns
original_revenue_sum = transaction_data['Revenue'].sum()
adjusted_revenue_sum = adjusted_original_transaction['Revenue'].sum()

original_revenue_sum, adjusted_revenue_sum


(763434.0, 655178.1)

In [6]:
adjusted_original_transaction[adjusted_original_transaction['Revenue'] == 0]

,Period,Treatment,Revenue,Expense,Customer ID
16,2025-01-06,051,0.0,125.0,Patient 517
71,2025-01-21,044,0.0,62.5,Patient 488
102,2025-01-30,055,0.0,62.5,Patient 726
146,2025-02-14,041,0.0,125.0,Patient 645
151,2025-02-15,091,0.0,62.5,Patient 417
...,...,...,...,...,...
1414,2025-11-15,051,0.0,125.0,Patient 474
1467,2025-12-08,054,0.0,125.0,Patient 211
1484,2025-12-12,085,0.0,62.5,Patient 36
1516,2025-12-25,086,0.0,125.0,Patient 585


In [7]:
# Revised ModelFairCredit class with the same class and variable names
import pandas as pd
import numpy as np
import numpy_financial as npf

class ModelFairCredit:
    def __init__(self, transaction_data, minimum_allowable_price=500, applicant_ratio=0.5, period_term=10, monthly_interest=0.01, percentage_downpayment=0.3):
        self.transaction_data = transaction_data
        self.minimum_allowable_price = minimum_allowable_price
        self.applicant_ratio = applicant_ratio
        self.period_term = period_term
        self.monthly_interest = monthly_interest
        self.percentage_downpayment = percentage_downpayment
        
        self.aggregated_amortization_schedule = pd.DataFrame()
        
    def generate_amortization_schedule(self):
        adjusted_original_transaction = self.transaction_data.copy()
        adjusted_original_transaction['Adjusted'] = False  # Add a flag to track adjustments

        for _, row in self.transaction_data.iterrows():
            if row['Revenue'] < self.minimum_allowable_price:
                continue
            
            if np.random.rand() > self.applicant_ratio:
                continue

            amortization_schedule = pd.DataFrame()
            amortization_schedule['Period'] = pd.date_range(start=row['Period'], periods=self.period_term, freq='30D')
            
            price_as_principal = row['Revenue'] * (1 - self.percentage_downpayment)
            monthly_payment = npf.pmt(self.monthly_interest, self.period_term, -price_as_principal)
            
            amortization_schedule['Revenue'] = monthly_payment
            amortization_schedule['Expense'] = 0
            amortization_schedule['Customer ID'] = row['Customer ID']
            
            # Adjust the revenue for the specific row only
            mask = (
                (adjusted_original_transaction['Customer ID'] == row['Customer ID']) &
                (adjusted_original_transaction['Revenue'] == row['Revenue']) &
                (~adjusted_original_transaction['Adjusted'])  # Only unadjusted rows
            )
            if mask.any():
                adjusted_original_transaction.loc[mask, 'Revenue'] = row['Revenue'] * self.percentage_downpayment
                adjusted_original_transaction.loc[mask, 'Adjusted'] = True

            # Concatenate the amortization schedule to the aggregated schedule
            self.aggregated_amortization_schedule = pd.concat(
                [self.aggregated_amortization_schedule, amortization_schedule], ignore_index=True
            )
        
        # Drop the 'Adjusted' flag before returning
        adjusted_original_transaction = adjusted_original_transaction.drop(columns=['Adjusted'])
        return self.aggregated_amortization_schedule, adjusted_original_transaction
        
    def group_by_period(self):
        # Return after grouping by month-year
        return self.aggregated_amortization_schedule.groupby(self.aggregated_amortization_schedule['Period'].dt.to_period("M")).agg({'Revenue':'sum', 'Expense':'sum'}).reset_index()



In [11]:
transaction_data = pd.read_csv('sample_transaction_data.csv')

model = ModelFairCredit(transaction_data)
aggregated_amortization_schedule, adjusted_original_transaction = model.generate_amortization_schedule()

# Check the sums of the Revenue columns
original_revenue_sum = transaction_data['Revenue'].sum()
adjusted_revenue_sum = adjusted_original_transaction['Revenue'].sum()

original_revenue_sum, adjusted_revenue_sum


(813134.0, 759337.6)

In [12]:
adjusted_original_transaction[adjusted_original_transaction['Revenue'] == 0]

,Period,Treatment,Revenue,Expense,Customer ID
